In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy import stats

## grille des sauts

In [2]:
def grille_sauts(T, kn,lamda):
    jumps_t = []
    grid = []
    x =  np.random.exponential(1/lamda)
    sum_x = x
    
    vec = []
    t = 0
    vec.append(t)
    vec.append(0)
    grid.append(vec)
    vec = []
    
    # construction of the jump times
    while sum_x < T:
        jumps_t.append(sum_x)
        x = np.random.exponential(1/lamda)
        sum_x += x
    
    i = 0
    # construction of the grid on the interval [0,last jump]
    jump_size = len(jumps_t)
    epsilon = 0
    while t < T and i < jump_size:
        epsilon = 0
        t = min(t + 1.0/kn, jumps_t[i])
        if t == jumps_t[i]:
            i += 1
            epsilon = 1 # it is a jump time
        
        vec.append(t)
        vec.append(epsilon)
        grid.append(vec)
        
        vec = []
    
    # to construct the grid on the inverval ]last jump, T]
    while t < T:
        t += 1.0/kn
        
        vec.append(t)
        vec.append(0) # epsilon = 0
        grid.append(vec)
        
        vec = []
    
    vec = []
    vec.append(T)
    vec.append(0)
    
    if grid[-1][0] < T:
        grid.append(vec)
    
    if grid[-1][0] > T:
        grid.pop()
        grid.append(vec)
    
    return grid


## call

In [3]:
def european_call(K,T,sig,r,S_t,t=0):
    d1=(1/(sig*np.sqrt(T-t)))*(np.log(S_t/K)+(r+((sig**2)/2))*(T-t))
    d2=d1-sig*np.sqrt(T-t)
    C=S_t*(norm.cdf(d1))-K*np.exp(-r*(T-t))*norm.cdf(d2)
    return C 

In [4]:
def normalCFD(value):
    return 0.5 * math.erfc(-value * math.sqrt(1/2))


def kou_doleans_dade_2_call(M, p, lambda_, lambda_plus, lambda_moins, strike, x0, T, c, r):
    result = []
    interm=[]
    b =r -c/2- lambda_*(p*(lambda_plus/(lambda_plus - 1)) + (1-p)*lambda_moins/(lambda_moins+1) -1 )
    sqrtc = math.sqrt(c)
    for j in range(1, M):
        saut = 0
        nombre_de_saut = np.random.poisson(lambda_)
        for i in range(nombre_de_saut):
            u=np.random.uniform(0,1)
            H = np.random.binomial(n=1, p=p)
            saut +=( H *(-math.log(u)/lambda_plus) + (1-H)* math.log(u)/lambda_moins)
        z=np.random.normal(0, 1)
        interm.append(max(0, x0 * math.exp(b + sqrtc * z +saut) - strike))
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result
def kou_doleans_dade_2_call_reducVar(M, p, lamda, lambd_plus, lambd_moins, strike, x0, T, c, r):
    result = []
    interm=[]

    b = r -c/2- lamda*(p*(lambd_plus/(lambd_plus - 1)) + (1-p)*lambd_moins/(lambd_moins+1) -1 )
    sqrtc = np.sqrt(c)
    
    for j in range(1, M):
        saut = 0
        nombre_de_saut = np.random.poisson(lamda)
        for i in range( nombre_de_saut):
            u=np.random.uniform(0,1)
            v=np.random.uniform(0,1)
            H = np.random.binomial(n=1, p=p)
            saut +=( H*(-math.log(u)/lambd_plus) + (1-H)*math.log(v)/lambd_moins)
            
        mu = x0 * np.exp(b + c/2 + saut)
        interm.append(european_call(strike,1,c,r,mu,t=0))
   
    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result
       


def merton_call_reducVar(M, lamda, delta, strike, x0, T, r, c):
    interm = []
    result = []
    b = r - c/2 + lamda * (1 - np.exp((delta**2)/2))
    sqrtc = np.sqrt(c)
    for j in range(1, M):
        realisation_normale = 0
        N_T=np.random.poisson(lamda)
        for i in range( N_T):         
            z=np.random.normal(0, 1)
            realisation_normale += z

        mu = x0 * np.exp(b + c/2 + delta * realisation_normale)
        mu_1 = x0 * np.exp(b + c/2 - delta * realisation_normale)
        interm.append((european_call(strike,1,c,r,mu,t=0)+european_call(strike,1,c,r,mu_1,t=0))/2)

    #result.append(np.mean(interm))
    #result.append(1.96*np.std(interm)/np.sqrt(M))
    result = { 'mean': np.mean(interm), 'std': np.std(interm), 
               'lower': np.mean(interm) - 1.96*np.std(interm)/np.sqrt(M), 
               'upper': np.mean(interm) + 1.96*np.std(interm)/np.sqrt(M) }
    return result

In [5]:
b = 0.01
c = 0.04
lambda_moins=6
lambda_=4
lambda_plus=3
p=0.5
x0=100
strike=[80,100,120]
T=1
kn=1000
M=10000
r=0.02
delta=0.2


In [6]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_2_call(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80
{'mean': 47.30660123471908, 'std': 275.1663081691987, 'lower': 41.913341594602784, 'upper': 52.69986087483538}
strike: 100
{'mean': 37.99287770284061, 'std': 211.1487593259265, 'lower': 33.85436202005245, 'upper': 42.13139338562877}
strike: 120
{'mean': 32.74910699630464, 'std': 212.51243845411057, 'lower': 28.58386320260407, 'upper': 36.91435079000521}


In [7]:
for s in strike:
    print('strike:',s)
    print(kou_doleans_dade_2_call_reducVar(M, p, lambda_, lambda_plus, lambda_moins, s, x0, T, c, r))

strike: 80
{'mean': 49.01937409099591, 'std': 256.29376822591115, 'lower': 43.99601623376805, 'upper': 54.04273194822377}
strike: 100
{'mean': 32.80623478416636, 'std': 130.27353996861055, 'lower': 30.25287340078159, 'upper': 35.359596167551125}
strike: 120
{'mean': 30.057715767266213, 'std': 151.53152573380538, 'lower': 27.08769786288363, 'upper': 33.0277336716488}


In [8]:
for s in strike:
    print('strike:',s)
    print(merton_call_reducVar(M, lambda_, delta, s, x0, T, r, c))

strike: 80
{'mean': 28.742739185410173, 'std': 19.892373566342844, 'lower': 28.352848663509853, 'upper': 29.132629707310493}
strike: 100
{'mean': 17.449495009158575, 'std': 20.577570532409617, 'lower': 17.046174626723346, 'upper': 17.852815391593804}
strike: 120
{'mean': 10.61930203158278, 'std': 19.612372160638465, 'lower': 10.234899537234266, 'upper': 11.003704525931294}
